In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)

In [2]:
df_questions = pd.read_hdf("auto_tagging_data_v2.h5")

In [3]:
df_questions.head()

,Id,Title,Body,Tags
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning]
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting]
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian]
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]"
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation]


In [4]:
df_questions['Text'] = df_questions["Title"] + " " + df_questions["Body"]

In [7]:
df_questions.head()

,Id,Title,Body,Tags,Text
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning],"The Two Cultures: statistics vs. machine learning? <p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statisti..."
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting],Forecasting demographic census <p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census ...
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian],Bayesian and frequentist reasoning in plain English <p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]","What is the meaning of p values and t values in statistical tests? <p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk b..."
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation],"Examples for teaching: Correlation does not mean causation <p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustra..."


In [8]:
def clean_text(text):
    # remove html tags and url links
    text = re.sub(r'<.*?>', '', text)
    # remove everything alphabets
    text = re.sub("[^a-zA-Z]"," ",text)
    # remove whitespaces
    text = ' '.join(text.split())
    
    return text

In [9]:
df_questions['Text'] = df_questions['Text'].apply(lambda x: clean_text(x))
df_questions['Text'] = df_questions['Text'].str.lower()

In [10]:
df_questions[['Id', 'Text', 'Tags']].sample(5)

,Id,Text,Tags
32538,215389,logistic regression with temporal autocorrelated data my objective is to simulate daily temporal series of wet dry sequence of years days i m using a logistic regression with one continuous covari...,"[regression, logistic, generalized-linear-model, autocorrelation]"
47262,70517,weighted sample and logistic regression i am doing logistic regression in which i have a weighted sample data should i should have the weight option on or off when doing logistic regression,"[logistic, spss]"
49067,167357,using percentage values in anova test i have a number of values expressed as an index the proportion of heart muscle cells in s phase compared to the total population of heart muscle cells calcula...,"[statistical-significance, anova]"
46437,35407,how do i estimate a repeated measures ancova in r and spss i m currently running a perception experiment dv error this in degrees how much an observer was away from the real answer iv the time bin...,"[r, spss, repeated-measures]"
9426,45348,determining conserved features using a bayesian approach i would like to perform some sort of binary classification and my data set consists of examples for each class which are vectors with eleme...,"[probability, bayesian, classification, feature-selection]"


In [12]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_questions['Text'])

In [14]:
len(tokenizer.word_index)

81956

In [26]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

81957

In [15]:
sequences = tokenizer.texts_to_sequences(df_questions['Text'])

In [16]:
i = 0
print(df_questions['Text'][i], '\n'), print(sequences[i])

the two cultures statistics vs machine learning last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to this simon blomberg from r s fortunes package to paraphrase provocatively machine learning is statistics minus any checking of models and assumptions brian d ripley about the difference between machine learning and statistics user vienna may season s greetings andrew gelman in that case maybe we should get rid of checking of models and assumptions more often then maybe we d be able to solve some of the problems that the machine learning people can solve but we can t there was also the statistical modeling the two cultures paper by leo breiman in which argued that statisticians rely too heavily on data modeling and that machine learning techniques are making progress by instead relying on the predictive accuracy of models has the statistics field 

(None, None)

In [17]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))

In [18]:
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  97.0
40th percentile:  116.0
50th percentile:  137.0
60th percentile:  162.0
70th percentile:  193.0
80th percentile:  238.0
90th percentile:  320.0
95th percentile:  411.0
99th percentile:  678.0


In [19]:
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions['Tags'])
y = multilabel_binarizer.transform(df_questions['Tags'])

In [21]:
padded_seq.shape, y.shape

((76365, 125), (76365, 100))

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y,test_size=0.2,random_state=9)

In [23]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [24]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [27]:
model = Sequential()
model.add(Embedding(vocab_size +1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 5, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 125, 128)          10490624  
                                                                 
 dropout (Dropout)           (None, 125, 128)          0         
                                                                 
 conv1d (Conv1D)             (None, 121, 300)          192300    
                                                                 
 global_max_pooling1d (Globa  (None, 300)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 100)               30100     
                                                                 
Total params: 10,713,024
Trainable params: 10,713,024
Non-trainable params: 0
__________________________________________

In [29]:
callbacks = [
             EarlyStopping(patience=3),
             ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)
            ]

In [31]:
history = model.fit(x_train, y_train,epochs=5,batch_size=254,validation_split=0.1,callbacks=callbacks)

Epoch 1/5
217/217 [==============================] - 85s 391ms/step - loss: 0.0695 - accuracy: 0.2357 - val_loss: 0.0656 - val_accuracy: 0.2745
Epoch 2/5
217/217 [==============================] - 86s 397ms/step - loss: 0.0614 - accuracy: 0.2978 - val_loss: 0.0602 - val_accuracy: 0.3008
Epoch 3/5
217/217 [==============================] - 88s 404ms/step - loss: 0.0564 - accuracy: 0.3315 - val_loss: 0.0571 - val_accuracy: 0.3295
Epoch 4/5
217/217 [==============================] - 87s 402ms/step - loss: 0.0529 - accuracy: 0.3553 - val_loss: 0.0550 - val_accuracy: 0.3452
Epoch 5/5
217/217 [==============================] - 89s 408ms/step - loss: 0.0501 - accuracy: 0.3737 - val_loss: 0.0541 - val_accuracy: 0.3574


In [32]:
preds = model.predict(x_val)
preds.shape

478/478 [==============================] - 7s 14ms/step


(15273, 100)

In [33]:
preds_int = (preds >= 0.45).astype(int)

In [34]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_score(y_val, preds_int, average="micro")

0.4782484608798467

In [35]:
def infer_tags(q):
    q = clean_text(q)
    q = q.lower()
    q_seq = tokenizer.texts_to_sequences([q])
    q_seq_padded = pad_sequences(q_seq, maxlen=125)
    q_pred = model.predict(q_seq_padded)
    q_pred = (q_pred >= 0.3).astype(int)
    
    return multilabel_binarizer.inverse_transform(q_pred)

In [36]:
new_q = "Regression line in ggplot doesn't match computed regression Im using R and created a chart using ggplot2. I then create a regression so I can make some predicitions I pass my data frame of to the predict function predict(regression, Measures) I'd expect the predictions to be the same as if I used the regression line on the chart, but they aren't the same. Why would this be the case? Is there a setting in ggplot or is my expectation incorrect?"

# get tags
infer_tags(new_q)

1/1 [==============================] - 0s 19ms/step


[('r', 'regression')]